In [1]:
from jupyter_dash import JupyterDash
import dash_html_components as html
import dash_bootstrap_components as dbc
import dash_core_components as dcc
from dash.dependencies import Output, Input

app = JupyterDash(__name__,
					external_stylesheets=[dbc.themes.BOOTSTRAP])

In [7]:
import pandas as pd
from pathlib import Path
from functools import lru_cache

@lru_cache()
def get_poverty_data_fast():
	data_path = Path('../data/PovStatsData.csv')
	# csv_name = data_path.glob('*poverty.csv')
	df = pd.read_csv(data_path)
	
	return df

poverty_data = get_poverty_data()

In [4]:
@app.callback(Output('report', 'children'),
			Input('country', 'value'))
def display_country_report(country):
	if country is None:
		return ''
	filtered_df = poverty_data[(poverty_data['Country Name'] == country) &
							(poverty_data['Indicator Name']=='Population, total')]
	population= filtered_df.loc[:, '2010'].values[0]
	return [html.H3(country), f'The population of {country} in 2010 was {population:,.0f}.']

In [5]:

app.layout = html.Div(children=[
    html.H1('Poverty And Equity Database',
            style={'color': 'blue',
                   'fontSize': '40px'}),
    html.H2('The World Bank'),
    html.P("Key Facts:"),
    dbc.Tabs([
        dbc.Tab([
            html.Ul([
                html.Br(),
                html.Li("Number of Economies: 170"),
                html.Li("Temporal Coverage: 1974 - 2019"),
                html.Li('Last Update: March 18, 2020'),
                html.Li([
                    'Source: ',
                    html.A('https://datacatalog.worldbank.org/dataset/poverty-and-Equity',
                           href='https://datacatalog.worldbank.org/dataset/poverty-and-equity-database'),
                ]),


            ]),
        ], label='Key Facts'),
        dbc.Tab([
            html.Ul([
                html.Br(),
                html.Br(),
                html.Li(
                    "Book title: Interfactive Dashboards and Data Apps with plotly Dash"),
                html.Li(['GitHub repo: ',
                         html.A('https://github.com/PacktPublishing/Interactive-Dashboard',
                                href='https://github.com/PackPublishing/Interactive-Dashboarding'),
                         ])
            ])
        ], label='Project Info'),
        dbc.Tab([
            html.Ul([
                html.Div([dcc.Dropdown(id='country',
                options=[{'label': country, 'value': country}
                    for country in poverty_data['Country Name'].unique()]
                         ),
                html.Div(id='report')
                ])
            ]),
            ], label='Poverty_data'),
])
])

In [6]:
app.run_server(mode='external', height=600, width='80%')

Dash app running on http://127.0.0.1:8050/
